# Assignment 2

## Roll number: 


### Instructions
 * Fill in the roll-number in the cell above.
 * Code must be submitted in Python in jupyter notebooks. We highly recommend using anaconda/miniconda distribution or at the minimum, virtual environments for this assignment.
 * All the code and result files should be uploaded in the github classroom.
 *  Most of the questions require you to **code your own functions** unless there is a need to call in the abilities of the mentioned libraries, such as Visualisation from Open3D. Make sure your code is modular since you will be reusing them for future assignments. All the functions related to transformation matrices, quaternions, and 3D projection are expected to be coded by you.
 *  All the representations are expected to be in a right-hand coordinate system.
<!--  * Answer to the descriptive questions should be answered in your own words. Copy-paste answers will lead to penalty. -->
 * You could split the Jupyter Notebook cells where TODO is written, but please try to avoid splitting/changing the structure of other cells.
 * All the visualization should be done inside the notebook unless specified otherwise.
 * Plagiarism will lead to heavy penalty.
 * Commit the notebooks in the repo and any other results files under the result folder in the GitHub Classroom repo. 
 * This is a group assignment. Discussions are encouraged but any sharing of code among different teams will be penalized. 

## SECTION 1: ICP with SVD

### 1.1 Perform Procrustes alignmenton two point clouds with (given) known correspondences. (5 Points)

Let X be your point cloud observed from the initial pose P1. You then transform it to a new pose P2. Now you wish to apply ICP to recover transformation between (X & P1) and (X & P2).

Use toothless.ply point cloud and perform the alignment between the two point clouds using procrustes alignment. Your task is to write a function that takes two point clouds as input wherein the corresponding points between the two point clouds are located at the same index and returns the transformation matrix between them. Compute the alignment error after aligning the two point clouds.

<b>Use root mean squared error (RSME) as the alignment error metric.</b>

Make sure your code is modular as we will use this function in the next sub-part.

We will again use our own getTransform function to generate the 4x4 Transformation matrix to transform the pointcloud to P2, so make sure your code works for any general Transformation matrix

In [1]:
import numpy as np
import math

def alpha_rot(alpha):
    return np.array([
        [   1.0,    0.0,                0.0             ],
        [   0.0,    math.cos(alpha),   -math.sin(alpha) ],
        [   0.0,    math.sin(alpha),    math.cos(alpha) ]
        ],dtype=np.float32)

def beta_rot(beta):
    return np.array([
        [   math.cos(beta),     0.0,    math.sin(beta)  ],
        [   0.0,                1.0,    0.0             ],
        [  -math.sin(beta),     0.0,    math.cos(beta)  ]
        ],dtype=np.float32)

def gamma_rot(gamma):
    return np.array([
        [   math.cos(gamma),   -math.sin(gamma),    0.0 ],
        [   math.sin(gamma),    math.cos(gamma),    0.0 ],
        [   0.0,                0.0,                1.0 ]
        ],dtype=np.float32)
        
def euler2rm(alpha, beta, gamma):
    return np.matmul(alpha_rot(alpha),np.matmul(beta_rot(beta),gamma_rot(gamma)))

def getTransform():
    # Random rotation matrices for X, Y, and Z axes
    R = euler2rm(np.random.uniform(0, 2 * np.pi),
                 np.random.uniform(0, 2 * np.pi),
                 np.random.uniform(0, 2 * np.pi))
    
    # Random translation matrix
    T = np.array([
        [np.random.uniform(-500, 500)],
        [np.random.uniform(-500, 500)],
        [np.random.uniform(-500, 500)]
    ])
    
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = R
    transformation_matrix[:3, 3] = T.flatten()
    
    return transformation_matrix

In [2]:
T = getTransform()
def icp_known_correspondences(T):
    #### YOUR CODE HERE ####
    pass

### 1.2 Implement ICP algorithm with unknown correspondences. (5 Points)

Your task is to write a function that implements ICP and takes two point clouds as input wherein the correspondances are unknown. Visualize the pointclouds and plot their individual coordinate frames as you perform ICP over them. Compute the alignment error in each iteration. 

Refer to Shubodh's notes to compute correspondences: https://saishubodh.notion.site/Mobile-Robotics-Navigating-from-Theory-to-Application-0b65a9c20edd4081978f4ffad917febb?p=a25686ce1a11409d838d47bcac43ab4b&pm=s#bb9aaf2e316b4db3b399df1742f0444c


In [3]:
P = getTransform()
Q = getTransform()

def icp_unknown_correspondences(P, Q):
    #### YOUR CODE HERE ####
    pass

## SECTION 2: ICP with Lie Groups

### 2.1 Predict the Transformation matrix between 2 point clouds with known correspondences (15 Points)
Perform the same task as 1.1 using Lie Group Optimization from scratch to predict the transformation between the 2 point clouds.

Refer: https://saishubodh.notion.site/Mobile-Robotics-Navigating-from-Theory-to-Application-0b65a9c20edd4081978f4ffad917febb?p=ee55fe5689794693910ab7861bef067b&pm=s#7b82d84766a84b63b91d859579e4886b


In [10]:
import numpy as np
import open3d as o3d
import time

def skew(vector):
    x, y, z = vector.ravel()
    
    return np.array([
        [ 0, -z,  y],
        [ z,  0, -x],
        [-y,  x,  0]
    ])

def invert_transform(matrix):
    rotation_matrix = matrix[:3, :3]
    translation_vector = matrix[:3, 3]

    inv_rotation_matrix = np.transpose(rotation_matrix)
    inv_translation_vector = -np.matmul(inv_rotation_matrix, translation_vector)

    inv_matrix = np.eye(4)
    inv_matrix[:3, :3] = inv_rotation_matrix
    inv_matrix[:3, 3] = inv_translation_vector
    
    return inv_matrix

def transform_point_cloud(point_cloud, transformation_matrix, downsample=1.0):
    if downsample < 1.0:
        indices = np.arange(0, point_cloud.shape[0], int(1.0 / downsample))
        point_cloud = point_cloud[indices]

    rotated_points = np.dot(point_cloud, transformation_matrix[:3, :3].T)
    transformed_points = rotated_points + transformation_matrix[:3, 3]

    return transformed_points

def icp_with_lie(pcl0, pcl1, guess_transform, learning_rate):
    pcl1_transformed = transform_point_cloud(pcl1, guess_transform)
    
    residuals = (pcl1_transformed - pcl0).reshape(-1,1)

    jacobian = np.zeros((pcl1.shape[0] * 3, 6))

    for j in range(pcl1_transformed.shape[0]):
        jacobian[3 * j: 3 * j + 3, 0:3] = np.eye(3)
        jacobian[3 * j: 3 * j + 3, 3:6] = -skew(transform_point_cloud(pcl1[j],guess_transform))
    delta_x = -learning_rate * (np.linalg.pinv(jacobian.T @ jacobian) @ jacobian.T @ residuals)
    return delta_x, residuals

def se3_exp(xi):
    w = xi[3:].reshape(1, -1)
    v = xi[:3]
    theta = np.linalg.norm(w)
    axis = w/theta
    skew_axis = skew(axis)
    if theta < 1e-6:
        R = np.eye(3)
        J = np.eye(3)
    else:
        R = ((1-np.cos(theta))*(skew_axis@skew_axis)) + np.eye(3) + (np.sin(theta)*skew_axis)
        J = (((np.sin(theta)/theta))*np.eye(3)) + ((1-(np.sin(theta)/theta))*(axis.T@axis)) + (((1-np.cos(theta))/theta)*skew_axis)

    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = (J @ v).flatten()
    return T

pcl = o3d.io.read_point_cloud('data/toothless.ply')
pcl_array = np.array(pcl.points)

transform0 = getTransform()
transform1 = getTransform()

downsample = 0.001

pcl0 = transform_point_cloud(pcl_array, transform0, downsample)
pcl1 = transform_point_cloud(pcl_array + np.random.normal(0.0, 0, pcl_array.shape), transform1, downsample)

pcl0_geom = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pcl0))
frame0 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0, origin=[0, 0, 0])
pcl0_geom.paint_uniform_color([1.0, 0.0, 0.0])
pcl1_geom = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pcl1))
frame1 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0, origin=[0, 0, 0])
pcl1_geom.paint_uniform_color([0.0, 0.0, 1.0])

vis = o3d.visualization.Visualizer()
vis.create_window()

vis.add_geometry(pcl0_geom)
vis.add_geometry(frame0)
vis.add_geometry(pcl1_geom)
vis.add_geometry(frame1)

vis.poll_events()
vis.update_renderer()

initial_learning_rate = 9e-2
decay_rate = 0.985
min_learning_rate = 1e-2

opti_epsilon = np.random.uniform(0,0,(6, 1))
opti_epsilon[3] = 0
opti_epsilon[4] = 0
opti_epsilon[5] = 0
opti_transform = se3_exp(opti_epsilon)

print("Trasform 0:")
print(transform0)
print("Trasform 1:")
print(transform1)
print("Initial Transformation Guess:")
print(opti_transform)
print(f"Initial Learning Rate: {initial_learning_rate}")
print(f"Decay Rate: {decay_rate}")
print(f"Minimum Learning Rate: {min_learning_rate}")

for i in range(4000):
    current_learning_rate = max(initial_learning_rate * (decay_rate ** i), min_learning_rate)
    delta_epsilon, curr_residuals = icp_with_lie(pcl0, pcl1, opti_transform, current_learning_rate)
    opti_epsilon += delta_epsilon
    opti_transform = se3_exp(opti_epsilon)
    pcl1_geom.points = o3d.utility.Vector3dVector(transform_point_cloud(pcl1,opti_transform))
    frame1.vertices = o3d.utility.Vector3dVector(transform_point_cloud(np.array(frame0.vertices),opti_transform))
    vis.update_geometry(pcl1_geom)
    vis.update_geometry(frame1)
    vis.poll_events()
    vis.update_renderer()
    if np.linalg.norm(curr_residuals)/curr_residuals.shape[0] < 9e-2:
        pcl0_geom.paint_uniform_color([0.0, 1.0, 0.0])
        pcl1_geom.paint_uniform_color([0.0, 1.0, 0.0])
        vis.update_geometry(pcl0_geom)
        vis.update_geometry(pcl1_geom)
        vis.update_geometry(frame1)
        vis.poll_events()
        vis.update_renderer()
        print("Converged")
        print("Expected transformation matrix:")
        print(np.array_str(transform0 @ invert_transform(transform1), precision=2, suppress_small=True))
        print("Final transformation matrix:")
        print(np.array_str(opti_transform, precision=2, suppress_small=True))
        break

if np.linalg.norm(curr_residuals)/curr_residuals.shape[0] > 9e-2:
    print("Failed to converge")
vis.run()
vis.destroy_window()

/tmp/ipykernel_193057/3066014773.py:54: RuntimeWarning: invalid value encountered in divide
  axis = w/theta


Trasform 0:
[[-2.22306699e-02  6.69047713e-01  7.42886901e-01 -2.68954205e+02]
 [-9.99182224e-01 -3.99697758e-02  6.09675096e-03 -1.66235245e+02]
 [ 3.37720402e-02 -7.42143869e-01  6.69389188e-01  4.90166577e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Trasform 1:
[[   0.77726167   -0.59407151    0.20722769 -106.45002932]
 [   0.5156008     0.79017222    0.33133626  -54.60717063]
 [  -0.36058301   -0.15068822    0.92047435   78.56881938]
 [   0.            0.            0.            1.        ]]
Initial Transformation Guess:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Initial Learning Rate: 0.09
Decay Rate: 0.985
Minimum Learning Rate: 0.01
Converged
Expected transformation matrix:
[[  -0.26    0.76    0.59 -301.47]
 [  -0.75   -0.54    0.37 -305.21]
 [   0.61   -0.35    0.72  479.46]
 [   0.      0.      0.      1.  ]]
Final transformation matrix:
[[  -0.26    0.76    0.59 -302.04]
 [  -0.75   -0.54    0.37 -309.25]
 [   0.61   -0.35    0.

## SECTION 3: Pose Graph Optimization with G2O
### Objective (5 Points)
A robot is travelling in a oval trajectory. It is equipped with wheel odometry for odometry information and RGBD sensors for loop closure information. Due to noise in wheel odometry it generates a noisy estimate of the trajectory. Our task is to use loop closure pairs to correct the drift.

We pose this problem as a pose graph optimization problem. In our graph, poses are the vertices and constraints are the edges. 

References:

1.) Class notes: https://saishubodh.notion.site/G2O-Edge-Types-d9f9ff63c77c4ceeb84b1e49085004e3

2.) Cyrill Stachniss lecture: https://www.youtube.com/watch?v=uHbRKvD8TWg 

### Given: 
In practical scenarios, we'd obtain the following from our sensors after some post-processing:

1. Initial position
2. Odometry Contraints/Edges: This "edge" information tells us relative transformation between two nodes. These two nodes are consecutive in the case of Odometry but not in the case of Loop Closure (next point).
3. Loop Closure Contraints/Edges: Remember that while optimizing, you have another kind of "anchor" edge as you've seen in 1. solved example.

You have been given a text file named `edges.txt` (in `data/`) which has all the above 3 and it follows G2O's format (as explained in class, [link here](https://saishubodh.notion.site/G2O-Edge-Types-d9f9ff63c77c4ceeb84b1e49085004e3) ). The ground truth is `gt.txt`.

Install g2o as mentioned in `g2o.ipynb` and optimise `edges.txt`

In [5]:
### YOUR CODE HERE ###

### Evo (10 Points)
We need a measure of how good the trajectory is. The error/loss used earlier doesn't tell us much about how the trajectory differs from the ground truth. Here, we try to do just this - compute error metrics. Rather than computing these from scratch, we will just Evo - https://github.com/MichaelGrupp/evo/.

Look at the absolute pose error (APE) and relative pose error (RPE). What do they capture and how are they calculated (descriptive answer)? How do these metrics differ in methodology? Can we determine if the error is more along the x/y axis?

Answer the above questions and report errors for the obtained trajectory.

In [6]:
### YOUR CODE HERE ###